In [6]:
# from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import wandb
wandb.login(relogin=True)
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/kutsev/.netrc


In [9]:
# Replace 'username/project_name' with your specific project path
project_path = 'dri-ice/AK_tests'
# Get all runs from the project
runs = api.runs(path=project_path)
# # Define the start and end datetime for filtering
# start_datetime = datetime(2024, 5, 18, 1, 0, 0)  # 1 AM on May 19, 2024
# end_datetime = start_datetime + timedelta(days=1)  # 24 hours later
# # Define the filters for runs created starting from 1 AM on May 19, 2024, with config edit='ft'
# filters = {
#     "created_at": {
#         "$gte": start_datetime.isoformat(),
#         "$lt": end_datetime.isoformat()
#     },
#     "edit": "ft"
# }
# # Get runs with the specified filters
# runs = api.runs(path=project_path, filters=filters)
filter_dict = {
    "config.edit": "lora",
    "state": "finished",
    "created_at": {"$gte": "2024-05-18"}  # Use the correct date format
}
runs = api.runs(project_path, filters=filter_dict)
# List to store individual DataFrames
data_frames = []
# Iterate over each run and capture the config and summary metrics
for run in runs:
    config_frame = pd.DataFrame([run.config])
    summary_frame = pd.DataFrame([run.summary_metrics])
    combined_frame = pd.concat([config_frame, summary_frame], axis=1)
    data_frames.append(combined_frame)
# Concatenate all the individual DataFrames into a single DataFrame
all_runs_df = pd.concat(data_frames, ignore_index=True)
# Adjust pandas display settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set display width
# Display the DataFrame
all_runs_df

,lr,sym,tag,edit,rank,save,seed,wbit,dtype,wandb,wbits,device,method,dataset,nearest,unlearn,alg_name,compress,edit_set,new_eval,nsamples,percdamp,act_order,ckpt_path,groupsize,kl_factor,load_ckpt,lora_type,num_steps,save_ckpt,stats_dir,batch_size,lora_alpha,max_length,model_name,save_model,zero_point,compression,use_variant,edit_dataset,lora_dropout,quant_method,weight_decay,interventions,static_groups,eval_zero_shot,model_parallel,sparsity_ratio,target_modules,norm_constraint,number_of_edits,true_sequential,compression_dataset,PPL,wmdp_bio stderr,Generalization,FLOPs,_wandb,wmdp_bio,Rewrite accuracy,mmlu stderr,Average bits,_step,PPL edits,wmdp_cyber,Local recall,Latency,_timestamp,Success recall,wmdp_bio accuracy,PPl edits unmasked,mmlu,Metrics,mmlu accuracy,wmdp_cyber stderr,wmdp_cyber accuracy,PPl QA,Locality,_runtime,Generalization recall,prune_method,sparsity_type
0,0.005,True,GPTQ8bit-to-lora,lora,8,out/,1,8,torch.float16,online,4.0,0,quant,c4,False,none,LoRA,True,1,False,128.0,0.01,False,/scratch/sux7mp/saved_models/checkpoint_202405...,128.0,0,False,adalora,70,False,/scratch/sux7mp/stats,50,32,30,meta-llama/Meta-Llama-3-8B,None,True,gptq,False,zsre,0,autogptq,0,"[compress, edit]",False,False,True,0.00,"[q_proj, v_proj]",False,50,False,c4,10.741209,0.013342,0.045889,-1,{'runtime': 6720},0.653574,0.140889,0.003936,4.250000,1,17056.351562,0.411676,0.023364,88.206251,1.716168e+09,0.140889,0.653574,4293.975586,0.565233,"{'ncols': 1, 'nrows': 13, 'sha256': '70ab931a2...",0.565233,0.011043,0.411676,4467.038574,0.023484,6721.256265,0.045889,NaN,NaN
1,0.005,True,GPTQ4bit-to-lora,lora,8,out/,1,4,torch.float16,online,4.0,0,quant,c4,False,none,LoRA,True,1,False,128.0,0.01,False,/scratch/sux7mp/saved_models/checkpoint_202405...,128.0,0,False,adalora,70,False,/scratch/sux7mp/stats,50,32,30,meta-llama/Meta-Llama-3-8B,None,True,gptq,False,zsre,0,autogptq,0,"[compress, edit]",False,False,True,0.00,"[q_proj, v_proj]",False,50,False,c4,10.741209,0.013342,0.045889,-1,{'runtime': 6686},0.653574,0.140889,0.003936,4.250000,1,17056.351562,0.411676,0.023364,87.801237,1.716168e+09,0.140889,0.653574,4293.975586,0.565233,{'artifact_path': 'wandb-client-artifact://6xn...,0.565233,0.011043,0.411676,4467.038574,0.023484,6686.885360,0.045889,NaN,NaN
2,0.005,True,AWQ8bit-to-lora,lora,8,out/,1,8,torch.float16,online,8.0,0,quant,c4,False,none,LoRA,True,1,False,128.0,0.01,False,/scratch/sux7mp/saved_models/checkpoint_202405...,128.0,0,False,adalora,70,False,/scratch/sux7mp/stats,50,32,30,meta-llama/Meta-Llama-3-8B,None,True,awq,False,zsre,0,autoawq,0,"[compress, edit]",False,False,True,0.00,"[q_proj, v_proj]",False,50,False,c4,12.323073,0.012980,0.674000,-1,{'runtime': 3107},0.688924,1.000000,0.003829,8.250000,1,311922.125000,0.459487,0.092327,88.398943,1.716164e+09,1.000000,0.688924,11900.338867,0.605683,"{'size': 247, '_type': 'table-file', 'ncols': ...",0.605683,0.011183,0.459487,5120.088379,0.091904,3107.301727,0.674000,NaN,NaN
3,0.005,True,GPTQ2bit-to-lora,lora,8,out/,1,2,torch.float16,online,4.0,0,quant,c4,False,none,LoRA,True,1,False,128.0,0.01,False,/scratch/sux7mp/saved_models/checkpoint_202405...,128.0,0,False,adalora,70,False,/scratch/sux7mp/stats,50,32,30,meta-llama/Meta-Llama-3-8B,None,True,gptq,False,zsre,0,autogptq,0,"[compress, edit]",False,False,True,0.00,"[q_proj, v_proj]",False,50,False,c4,10.741209,0.013342,0.045889,-1,{'runtime': 6689},0.653574,0.140889,0.003936,4.250000,1,17056.351562,0.411676,0.023364,87.288736,1.716168e+09,0.140889,0.653574,4293.975586,0.565233,{'artifact_path': 'wandb-client-artifact://gmb...,0.565233,0.011043,0.411676,4467.038574,0.023484,6689.302995,0.045889,NaN,NaN
4,0.005,True,AWQ4bit-to-lora,lora,8,out/,1,4,torch.float16,online,8.0,0,quant,c4,False,none,LoRA,True,1,False,128.0,0.01,False,/scratch/sux7mp/saved_models/checkpoint_202405...,128.0,0,False,adalora,70,False,/scratch/sux7mp/stats,50,32,30,meta-llama/Meta-Llama-3-8B,None,True,awq,False,zsre,0,autoawq,0,"[compress, edit]",False,False,True,0.00,"[q_p